# Problem set 2: part one

Total points: 20

Fraction of problem set two points: 25\%

**Concepts**: here, you're going to start using the SIP H-2A employer jobs and debarment data to practice exact matching,  regex, and fuzzy matching

In [124]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import recordlinkage


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Regex and exact matching (8 points)



# 1.1 Load data on debarments and job postings (0 points)

Load the following datasets stored in `pset2_part1_inputdata`
    
- Historical H2A debarments (debar.csv); call this `debar`
- Q1 2021 H2A job postings (jobs.csv); call this `jobs`


In [145]:
debar = pd.read_csv("debar.csv")
jobs = pd.read_csv("jobs.csv")

### 1.2 Try exact merge on business name  (2 points)

- Use the `EMPLOYER_NAME` field of jobs
- Use the `Name` field of debar

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are matches, print the name in each dataset, date range of debarment (`Start date` and `End date` in `debar`) and location from each data (`City, State` in `debar` and `EMPLOYER_CITY` and `EMPLOYER_STATE` in `jobs`)

**Resources**:
    - Slides 17-19 here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/slides/qss20_s21_class4.pdf 
    - Code here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/03_merging_session1.ipynb 


In [139]:
#debar.head()
#jobs.head()

merged = pd.merge(left = debar, right = jobs, how = "inner", left_on = "Name", right_on = "EMPLOYER_NAME")
print(merged[["Name", "EMPLOYER_NAME", "Start date", "End date", "City, State", "EMPLOYER_CITY", "EMPLOYER_STATE"]])

             Name   EMPLOYER_NAME Start date   End date       City, State  \
0  Rafael Barajas  Rafael Barajas  9/23/2016  9/22/2017  Sebring, Florida   

    EMPLOYER_CITY EMPLOYER_STATE  
0  Port St. Lucie             FL  


## 1.3 Targeted regex

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the name fields.


### 1.3.1 Converting to upper (2 points)

A. Convert the names `EMPLOYER_NAME` and `Name` to uppercase using list comprehension rather than df.varname.str.upper()

B. Print a random sample of 15 values of each result

C. Assign them back to the original data, writing over the original columns

**Resources**:
    - `new_colnames` line of code here for example of list comprehension: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/04_basicregex_formerging.ipynb
    - Sampling from a list without replacement using the `random` module: https://note.nkmk.me/en/python-random-choice-sample-choices/ 
    

In [146]:
debar["Name"] = [name.upper() for name in debar["Name"]]
jobs["EMPLOYER_NAME"] = [name.upper() for name in jobs["EMPLOYER_NAME"]]

print(random.sample(list(debar["Name"]), 15))
print(random.sample(list(jobs["EMPLOYER_NAME"]), 15))



['MYRKA MIREYA CARDENAS', 'YOLANDA CHAVEZ', 'OLD TREE FARMS/VERPAALEN CUSTOM SERVICE', 'LANDMARK LANDSCAPING', 'SAXTONS RIVER ORCHARDS, INC.', 'ROLLO FARM LABOR CONTRACTOR', 'VERN STRATTON FARMS', 'LABATTE FARMS', 'SRT FARMS', 'F&W FARMS', 'ANNABELLA LAND & CATTLE', 'DOVE CREEK FARMS', 'ANTON FERTILIZER INC.', 'SRT FARMS', 'OLD TREE FARMS/VERPAALEN CUSTOM SERVICE']
['CITRUS HARVESTING, INC.', 'TOP NOTCH PORK, LLC', 'ROJAS AND SIMON HARVESTING, LLC', 'WESTERN RANGE ASSOCIATION', 'WESLEY BLADOW', 'JEFFREY L. MANUEL', 'SESTER FARMS, INC', 'CITRUS HARVESTING, INC.', 'WESTERN RANGE ASSOCIATION', 'EAM HARVESTING LLC', 'STROOPE BEE COMPANY, LLC', 'ALCO HARVESTING, LLC', 'HUMPHREY COKER SEED CO', 'FAMILY FRESH HARVESTING LLC', 'AG LABOR LLC']


### 1.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a . but sometimes not

A. For each dataset, write a regex pattern to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example and print the result. The first positive example should return `CISCO PRODUCE INC`; the second positive example should return `AVOYELLES HONEY CO, LLC` the negative example should return `E.V. RANCH LLP` (so leave the dots between E.V. in)

C. Execute on (1) Name in `debar` (can just write over the col) and (3) EMPLOYER_NAME  in `jobs`, making sure to use the uppercase versions of the variables

**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content

**Resources**:
    - Regex slides are 22-40: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/slides/qss20_s21_class4.pdf 
    - Regex code: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/04_basicregex_formerging.ipynb

In [128]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [147]:
# Create Regex for punctuation removal
company_regex = r'(?<=INC)\.|(?<=LLC)\.|(?<=CO)\.'

#test it out
test_1 = re.sub(company_regex, '', pos_example_1)
test_2 = re.sub(company_regex, '', pos_example_2)
test_3 = re.sub(company_regex, '', neg_example)
print(test_1, test_2, test_3)

# apply using list comprehension
debar["Name"] = [re.sub(company_regex, "", company_name) for company_name in debar["Name"]]
jobs["EMPLOYER_NAME"] = [re.sub(company_regex, "", company_name) for company_name in jobs["EMPLOYER_NAME"]]

#check
debar.Name.head()
jobs.EMPLOYER_NAME.head()

CISCO PRODUCE INC AVOYELLES HONEY CO, LLC E.V. RANCH LLP


0                     J&J HARVESTING
1             STAHLMAN APIARIES, INC
2                      TRUST NURSERY
3               ANTON FERTILIZER INC
4    GREAT PLAINS FLUID SERVICE, INC
Name: Name, dtype: object

0    FAZIO FARMS OPERATING COMPANY, LLC
1                    CHARLIE SUNDERLAND
2                     MICHAEL RUDEBUSCH
3                          LODAHL FARMS
4                DUNSON HARVESTING, INC
Name: EMPLOYER_NAME, dtype: object

### 1.3.3 OPTIONAL extra credit - regex to separate companies from individuals (2 points)

You notice some employers in `debar` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned version of the Name in `debar`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string-- so in above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the and)
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
C. Iterate over the cleaned `Name` column in debar and execute the regex. Use it to create two new columns in debar:
    - A column for company (full Name string if left as is; pattern before COMPANY if one extracted)
    - A column for individual (full Name string if left as is; pattern before INDIVIDUAL if one extracted)
    
D. Print those columns for the rows containing the negative example and positive example

**Hint**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

For step C, you probably want to use a loop or function that uses if else to do different things if a match is found or not


In [ ]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

# 1.4 Fuzzy matching to match debarments to jobs (12 points)


### 1.4.1 Preprocessing location (2 points)

You want to block on state but notice that states in `debar` have a mix of two digit codes and full state names (e.g., GA versus Georgia) while states in `jobs` are all two-digit codes

A. Run the code below to load the `states` crosswalk. Use that crosswalk to create a field in `debar` that has the two-digit state abbreviation for all locations (hint: you may need to first split the string on the ", " or use str.replace to extract the state)

B. Use an `assert` statement to check that all the states in `debar` are two-digits after the cleaning

**Notes**: you can filter out states that are NaN

In [148]:
## code to load state crosswalk
## source- https://towardsdatascience.com/state-name-to-state-abbreviation-crosswalks-6936250976c


states = pd.read_csv("states_pset2.csv")
#states.head()

#get so the city, state column into a better name so it can be manipulated
debar.rename(columns={'City, State':'cityandstate'}, inplace=True)

#clear NA's
debar = debar.replace('nan', np.nan)
debar = debar.dropna(how = 'any')

#Use regex's to split up city and state into their own columns
debar["city"] = debar["cityandstate"].replace("\,.*","", regex = True)
debar["state"] = debar["cityandstate"].replace(".*\\s", "", regex = True)

#Iterate through the frame to see if we have an appropriate state code. If not,
#matches state full name into states dataframe and replaces with associated monicker.
for i in range(0, debar["state"].shape[0]):
    check = debar["state"].iloc[i]
    if len(check) != 2:
        for j in range(0, states["Description"].shape[0]):
            if check == states["Description"].iloc[j]:
                debar["state"].iloc[i] = states["Code"].iloc[j]
                
debar["state"].value_counts()

assert all(debar.state.str.len() == 2)

GA          19
TX          18
KS          10
FL          10
UT           6
KY           5
SD           5
CA           4
CO           3
NY           3
LA           3
AR           3
ND           3
Dakota       3
MA           3
MT           2
OK           2
ID           2
VT           1
ME           1
MD           1
Carolina     1
MN           1
SC           1
IL           1
TN           1
AK           1
Name: state, dtype: int64

AssertionError: 

In [149]:

#same rename process with a different removal process for NA
jobs.rename(columns={'EMPLOYER_POC_STATE':'state'}, inplace=True)
jobs = jobs[~jobs.state.isnull()]


In [157]:

#drop duplicates
jobs = jobs.drop_duplicates(subset= ["EMPLOYER_NAME"])
debar = debar.drop_duplicates(subset = ["Name"])

### 1.4.2 step by step fuzzy matching (4 points)



A. Write fuzzy matching code (don't yet put inside a user-defined function) that:

- Blocks on two-digit state code
- Finds matches based on similarity between the employer name (`Name`) in `debar` (uppercase and cleaned) and `EMPLOYER_NAME` in `jobs` (uppercase and cleaned). You can choose which distance metric and threshold to use (feel free to set a threshold low enough to get some matches even if that leads to some false positives).

For the steps after compute, just take any match with non-zero value rather than using a classifier (so skip the k-means or e-m step in the example code)

B. Print the matches and comment on examples of ones that seem like true positives and ones that seem like false matches

**Resources**:

- Solutions code here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/05_merging_session2_activitysolutions.ipynb 
- Example code here: https://github.com/rebeccajohnson88/qss20_slides_activities/blob/main/activities/05_merging_session2_codeexample.ipynb


**Hint**: you may need to deduplicate multiple records in the datasets for the recordlinkage package to work. See drop_duplicates within pandas and subset command

In [153]:
import nltk
from pyjarowinkler import distance
import datetime
from datetime import datetime

In [150]:
jobs.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"FAZIO FARMS OPERATING COMPANY, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,CHARLIE SUNDERLAND,Panter & Sunderland Nursery,...,N,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,MICHAEL RUDEBUSCH,NaN,...,N,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,LODAHL FARMS,NaN,...,Y,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"DUNSON HARVESTING, INC","Dunson Harvesting, Inc.",...,Y,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4


In [171]:
debar_samp = debar.sample(n = 50, random_state = 556)
jobs_samp = jobs.sample(n = 50, random_state = 556)

debar_samp["Name"].head()
## create blocker
link_debar_jobs = recordlinkage.Index()

debar_samp = debar_samp.reset_index()
jobs_samp = jobs_samp.reset_index()
##block on state code
link_debar_jobs.block("state")

#compute candidate links
candidate_links_state = link_debar_jobs.index(debar_samp, jobs_samp)

#compare class creation
compare = recordlinkage.Compare()
compare.string('Name', 'EMPLOYER_NAME', method = 'jarowinkler', threshold = 0.7)
compare.string('state', 'state', method = 'jarowinkler', threshold = 0.7)

#Feed candidate links to the compare class and compute
compare_vectors = compare.compute(candidate_links_state, debar_samp, jobs_samp)
compare_vectors 
               
## see under the hood steps to modif 

ecm = recordlinkage.ECMClassifier()
predicted_matches_ecm = pd.DataFrame(list(ecm.fit_predict(compare_vectors)), columns = ["debar_index", "jobs_index"])

print(predicted_matches_ecm)

debar_samp['debar_index'] = debar_samp.index

m1_add_debar = pd.merge(predicted_matches_ecm, debar_samp[["Name", "debar_index", "state"]], on = "debar_index", how = "inner")

jobs_samp["jobs_index"] = jobs_samp.index
m2_add_jobs = pd.merge(m1_add_debar, jobs_samp[["EMPLOYER_NAME", "jobs_index", "state"]], 
                       on = "jobs_index", how = "inner", suffixes = ["_debar", "_jobs"])
m2_add_jobs
                                                

90    RUBEN RUIZ (DESOTO HARVESTING)
11             LOEWEN HARVESTING LLC
76                  ROBERT D. TOWLES
20              SLASH E.V. RANCH LLP
83                 CIRA CORTEZ LOPEZ
Name: Name, dtype: object

<Index>

<Compare>

<Compare>

0    1
0  16  0.0  1.0
   23  0.0  1.0
   40  0.0  1.0
   42  0.0  1.0
11 16  0.0  1.0
...    ...  ...
49 45  0.0  1.0
46 7   0.0  1.0
   21  0.0  1.0
   33  0.0  1.0
   37  0.0  1.0

[120 rows x 2 columns]

   debar_index  jobs_index
0           44          49
1           48          49


,debar_index,jobs_index,Name,state_debar,EMPLOYER_NAME,state_jobs
0,44,49,PETER PETERS,TX,ETTER FARMS,TX
1,48,49,DOVE CREEK FARMS,TX,ETTER FARMS,TX


In [ ]:
#Neither of these seem to be exact matches. Frustrating, certainly. 
#I have also tried ramping the filter down to a 50% threshold, and the matches are even less certain. 

### 1.4.3 write a fuzzy matching function (6 points)

You want to see how the matches change if you add the town and not only state as a field and also want to automate the process of matching a bit to try different distance thresholds.

A. Extract the City from the `City, State` column of debar

B. Convert that new `city` column to uppercase and convert the `EMPLOYER_CITY` column in jobs to uppercase

C. Write a function surrounding the code in `recordlinkage` (so you don't need to recode the package from scratch) that (1) takes in each dataset, (2) blocks on two-digit state, and (3) fuzzy matches on employer name and employer city

D. Execute the function with a couple different string distance thresholds and print the results of each

5 out of 6 points: function takes arguments for input datasets, varname to block on, two varnames to fuzzy match on, string distance function, and string distance threshold
    
6 out of 6: above but function is also general enough that it takes a variable # of strings to match on--- so should work if you either execute just using employer name or also work if you execute using employer name and employer city

In [174]:
## Already created a city column above for part A 

## city to upper

debar["city"] = [city.upper() for city in debar["city"]]

def matcher(df1, df2, block, fuz1, fuz2, distance, thresh):
    df1_samp = df1.sample(n = 50, random_state = 556)
    df2_samp = df2.sample(n = 50, random_state = 556)
    
    linker = recordlinkage.Index()

    df1_samp = df1_samp.reset_index()
    df2_samp = df2_samp.reset_index()
    ##block on state code
    link_debar_jobs.block(block)
    
    candidate_links_state = linker.index(df1_samp, df2_samp)

#compare class creation
    compare = recordlinkage.Compare()
    compare.string(fuz1, fuz1, method = 'jarowinkler', threshold = thresh)
    compare.string(fuz2, fuz2, method = 'jarowinkler', threshold = thresh)
    
    compare_vectors = compare.compute(candidate_links_state, df1_samp, df2_samp)
    
    ecm = recordlinkage.ECMClassifier()
    predicted_matches_ecm = pd.DataFrame(list(ecm.fit_predict(compare_vectors)), columns = ["df1_index", "df2_index"])
    
    
    df1_samp['df1_index'] = df1_samp.index

    m1_add_df1 = pd.merge(predicted_matches_ecm, df1_samp[["Name", "df1_index", "state", "city"]], on = "df1_index", how = "inner")


    df2_samp["df2_index"] = df2_samp.index
    m2_add_df2 = pd.merge(m1_add_df1, df2_samp[["EMPLOYER_NAME", "df2_index", "state", "city"]], 
                       on = "df2_index", how = "inner", suffixes = ["_df1", "_df2"])

    return m2_add_jobs
                   

    
    
